In [1]:
# the Echo State Network is a specific architecture in Reservoir Computing
# I implemented this by following https://martinuzzifrancesco.github.io/posts/a-brief-introduction-to-reservoir-computing/

import sys; sys.path.append("../models")
import utils
import ESN

import xarray as xr
import numpy as np

from sklearn.metrics import r2_score, mean_squared_error

In [2]:
# note: the aim is for all preprocessing / reshaping to be folded into SimDataset

data_home = "/mnt/g/My Drive/GTC/solodoch_data_minimal"
lats = ["26N", "30S", "55S", "60S"]

lat = lats[0]
data = xr.open_dataset(f"{data_home}/{lat}.nc")

# apply whatever preprocessing we want *before* calling reshape_inputs
pp_data = utils.apply_preprocessing(data,
                                    mode="inputs",
                                    remove_season=True,
                                    remove_trend=True,
                                    standardize=True,
                                    lowpass=True)

# reshape as desired and convert to a numpy array
pp_data = utils.reshape_inputs(pp_data, keep_coords=["time"], return_pt=False)

# dummy strength data
strength = np.random.rand(pp_data.shape[0])
X = pp_data; y = strength

axes: ['time', 'feature']
variables: ['SSH', 'SST', 'SSS', 'OBP', 'ZWS']
shape: (288, 5)


In [3]:
data = np.append(X, y.reshape(-1, 1), axis=1)
# one step prediction
targets = data[1:, :]
data = data[:-1, :]

model = ESN.ESN(n_input=data.shape[1], n_reservoir=data.shape[1]*3)
model.fit(data, targets)
pred = model.predict(data)
r2 = round(r2_score(targets, pred), 3)
mse = round(mean_squared_error(targets, pred), 3)
print(f"R^2: {r2}")
print(f"MSE: {mse}")


R^2: 0.379
MSE: 0.186
